In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 125

import seaborn as sns

import pandas as pd
import numpy as np
import regex as re
import time 
from tqdm.notebook import tqdm
tqdm.pandas()


import requests
from bs4 import BeautifulSoup



In [2]:
## 1. Collecting data

# part 1
dir_1 = "C:\Users\thanh\OneDrive\Documents\HAN edu\minor MDD\project\scrapped data"
def single_request_scraping(index = 1, limit_retry = 25, sleep_time = 0.05):
    '''
    Collect news pages in the nltimes newspaper.
    Collect 20 messages each time. Use the index to crawl consecutive pages.
    With index = 1 get 20 latest news pages.

    '''
    allow_status = [200, 301]
    s = requests.Session()
    s.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    url = f"https://nltimes.nl/?page={index}"

    # Get the list of news_items
    try:
        response = s.get(url)
        try_left = limit_retry
        while (response.status_code not in allow_status and try_left > 0):
            print(f"Error {response.status_code} at {index}")
            response = s.get(url)
            try_left -= 1
        html_text = response.text
    except:
        print(f"Request error at {index}")
        return None
    
    html_tree = BeautifulSoup(html_text, 'html.parser')
    news_items = html_tree.findAll('div', class_='news-card col-lg-4 col-sm-6 col-xs-12')
    
    # Check for errors if can't get any items
    if (len(news_items) == 0):
        print(f'Page {index} got no item.')
        print(html_text)
        return None
    elif (len(news_items) != 21):
        print(f"Warning: Page {index} only got {len(news_items)} items.")
    
    # Get link, title, time and category from news_items
    raw_data = pd.DataFrame(columns=["links","title","time","content","class"])
    for item in news_items:
        try:
            title = item.find('div', class_='news-card__title').text.replace('\n','')
        except:
            title = ""
        link = item.find('a').attrs["href"]
        try:
            category = item.find('div', class_='news-card__categories').text
        except:
            category = ""
        try:
            time = item.find('div', class_='news-card__date').text
        except:
            time = ""
        raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)

    
    # Proceed to get the content of each link
    for _, row in raw_data.iterrows():
        try:
            response = s.get(row["links"])
            try_left = limit_retry
            while (response.status_code not in allow_status and try_left > 0):
                print(f"Error {response.status_code} link {row['links']}")
                response = s.get(row["links"])
                try_left -= 1
            news_page = response.content
        except:
            print(f"Request error link {row['links']}")
            row["content"] = ""
            continue
            
        news_tree = BeautifulSoup(news_page, "html.parser")
        # Get content
        try:
            content = news_tree.find('body').find_all('p')
            unwanted = ['Reporting by ANP', '© 2012-2023, NL Times, All rights reserved.']
            for x in list(dict.fromkeys(content)):
                if x.text.strip() not in unwanted:
                    row["content"] += x.text
        except:
            row["content"] = ''
    
    return raw_data

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3147780865.py, line 4)

In [ ]:
#Batch scraping
def batch_scraping(num = 150, output_dir = ""):
    '''
    Collect all data on nltimes.nl
    
    Param:
        num: number of request pages for 1 batch
    '''
    iter_num = 0  # Start batch number
    continue_flag = True # The loop termination flag when an error occurs

    while (continue_flag):
        
        '''
        Initialize empty dataframe.
        Then get enough pages for 1 batch.
        Then export the csv . file.
        '''
        batch_df = pd.DataFrame(columns=["links","title","time","content","class"])
        for index in range(iter_num*num+1,(iter_num+1)*num+1):
            data = single_request_scraping(index)
            if (data is None):
                continue_flag = False
                break
            print(f"Page {index} complete!")
            batch_df = batch_df.append(data)
        batch_df.to_csv(output_dir + f'crawling_{iter_num}.csv',index=False,encoding="utf-8")    
        iter_num+=1

In [ ]:
batch_scraping(output_dir = dir_1)

C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\582331683.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)
C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\582331683.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)
C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\582331683.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)
C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\582331683.py:64: Fut

Request error link /2023/01/08/foreign-minister-hoekstra-summons-iranian-ambassador-executions
Request error link /2023/01/08/many-people-still-looking-alleged-nazi-treasure-near-ommeren
Request error link /2023/01/08/train-traffic-zwolle-meppel-back-track-repairs
Page 1 complete!
Request error link /2023/01/08/foreign-minister-hoekstra-summons-iranian-ambassador-executions
Request error link /2023/01/08/many-people-still-looking-alleged-nazi-treasure-near-ommeren
Request error link /2023/01/08/train-traffic-zwolle-meppel-back-track-repairs


C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\3137663434.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  batch_df = batch_df.append(data)
C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\582331683.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)
C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\582331683.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)
C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\582331683.py:64: FutureWarning: The frame.append method is deprecated and will be removed from

Page 2 complete!
Request error link /2023/01/08/foreign-minister-hoekstra-summons-iranian-ambassador-executions
Request error link /2023/01/08/many-people-still-looking-alleged-nazi-treasure-near-ommeren
Request error link /2023/01/08/train-traffic-zwolle-meppel-back-track-repairs


C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\3137663434.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  batch_df = batch_df.append(data)
C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\582331683.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)
C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\582331683.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)
C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\582331683.py:64: FutureWarning: The frame.append method is deprecated and will be removed from

...

Request error link /2023/01/08/foreign-minister-hoekstra-summons-iranian-ambassador-executions
Request error link /2023/01/08/many-people-still-looking-alleged-nazi-treasure-near-ommeren
Request error link /2023/01/08/train-traffic-zwolle-meppel-back-track-repairs
Page 209 complete!


C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\3137663434.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  batch_df = batch_df.append(data)
C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\582331683.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)
C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\582331683.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append({"links":link, "time":time, "title":title, "class":category}, ignore_index=True)
C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\582331683.py:64: FutureWarning: The frame.append method is deprecated and will be removed from

Request error link /2023/01/08/foreign-minister-hoekstra-summons-iranian-ambassador-executions
Request error link /2023/01/08/many-people-still-looking-alleged-nazi-treasure-near-ommeren
Request error link /2023/01/08/train-traffic-zwolle-meppel-back-track-repairs
Page 210 complete!


C:\Users\thanh\AppData\Local\Temp\ipykernel_13728\3137663434.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  batch_df = batch_df.append(data)


OSError: Cannot save file into a non-existent directory: 'src\scraped_data'

In [ ]:
def batch_prepreprocess():
    total_files = 218
    text_attrs = ["title","time","content","class"]
    prev_df = None
    
    for index in tqdm(range(total_files)):
        df=pd.read_csv(f'src/scraped_data/crawling_{index}.csv')
        # Xóa dòng thiếu
        df.dropna(axis=0, how="any", inplace=True)
        # Xóa dòng trùng
        df.drop_duplicates(inplace=True)
        # Xóa dòng trùng với file trước
        if (prev_df is not None):
            df = df.merge(prev_df, how='outer', indicator=True).loc[lambda x : x['_merge'] == 'left_only']
            df.drop(['_merge'],axis=1,inplace=True)
        df.to_csv(f'src/scraped_data/crawling_{index}.csv', index=False)
        prev_df = df
        
    print("done.")

In [ ]:
batch_prepreprocess()

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html